In [5]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os
os.environ["OPENAI_API_KEY"] = ""

In [6]:
reader = PdfReader("C:/Users/SSTLabs/Desktop/여형구/설명서.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:1000]

'사용자 설명서HD928X2\n한국어\n목차\n주의  사항    3\n소개    6\n제품정보    7\n처음  사용  전   9\nNutriU 앱   9\n음성  컨트롤    9\n사용  준비    10\n제품  사용하기    10\n조리 표   10\n공기로 튀기기   12\n보온 모드 선택   15\n사전 설정으로 조리   16\n다른 사전 설정으로 변경   17\nNutriU 앱에서 레시피 시작하기   18\n홈메이드 감자 튀김 만들기   19\n세척    20\n세척 표   21\n보관    21\n재활용    21\n적합성  선언서    22\n보증  및 지원    22\n소프트웨어  업데이트    22\n장치  호환성    22\n공장  초기화    22\n문제  해결    233\n한국어\n주의 사항\n본 제품을 사용하기 전에 이 중요 정보를 주의 깊게 읽고 나중에 참조할 수 \n있도록 잘 보관하십시오.\n위험\n -항상 튀김 재료를 바구니에 넣어 가열장치와 \n접촉하지 않도록 주의하십시오. \n -제품이 작동하는 중에는 공기 흡입구와 배출구를 \n닫지 않도록 하십시오.\n -팬에 기름을 채우지 마십시오. 화재의 위험이 \n있습니다. \n -제품을 물 또는 다른 액체에 절대 담그거나 헹구지 \n마십시오.\n -제품에 물이나 기타 액체가 들어가지 않도록 \n하십시오. 감전 사고의 위험이 있습니다.\n -바구니의 최대 수량 표시선을 초과하는 양의 음식을 \n넣지 마십시오.\n -제품이 작동하는 중에는 제품 안쪽을 만지지 \n마십시오.\n -히터가 비어있고 히터 속에 음식물이 붙어있지는 \n않은지 항상 확인하십시오.\n경고\n -제품에 전원을 연결하기 전에, 제품에 표시된 전압과 \n사용 지역의 전압이 일치하는지 확인하십시오.\n -만약 플러그나 전원 코드 또는 제품이 손상되었을 \n경우에는 제품을 사용하지 마십시오.\n -전원 코드가 손상된 경우, 안전을 위해 필립스 \n서비스 센터 또는 필립스 서비스 지정점에 의뢰하여 \n교체하십시오.\n -어린이

In [7]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [8]:
summarize_document_chain.run(raw_text)

c:\Users\SSTLabs\miniconda3\envs\wahoman\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The HD928X2 user manual provides instructions for using an air fryer, including safety precautions, features, cleaning and maintenance, and warranty information. It also includes tips for cooking and using the accompanying app, as well as troubleshooting steps for common issues. The product is declared to comply with EU regulations and has a Wi-Fi indicator light.'

In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

c:\Users\SSTLabs\miniconda3\envs\wahoman\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
qa_document_chain.run(
    input_document=raw_text,
    question="홈메이드 감자튀김 만드는 방법은?")

'홈메이드 감자튀김을 만드는 방법은 다음과 같습니다:\n1. 감자를 껍질을 벗기고 씻어 깨끗이 세척합니다.\n2. 감자를 채썰거나 혹은 얇게 슬라이스하여 원하는 모양으로 자릅니다.\n3. 감자 조각들을 물에 담가 전분을 제거합니다. 이렇게 하면 감자튀김이 바삭해집니다.\n4. 감자를 물기를 털어내고 종이 타월 등으로 물기를 제거합니다.\n5. 감자 조각들을 뜨거운 기름에 튀겨 바삭하게 튀깁니다.\n6. 튀긴 감자튀김을 종이 타월 위에 올려 기름을 뺀 뒤 소금이나 기호에 맞는 양념을 뿌려 완성합니다.\n\n위의 방법을 따라 집에서 간단히 맛있는 홈메이드 감자튀김을 즐길 수 있습니다.'

In [11]:
qa_document_chain.run(
    input_document=raw_text,
    question="이순신의 아빠이름은?")

'죄송합니다, 그 질문에 대한 답변을 찾을 수 없습니다. 위 문서에서는 이순신의 아빠에 대한 정보가 포함되어 있지 않습니다. 다른 질문이 있으시면 도와드리겠습니다.'

In [15]:
import json
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

# 토큰화된 파일 위치
token_file_path = "C:/Users/SSTLabs/Desktop/여형구/LLM_RAG/tokens.json"

# 토큰 파일에서 데이터 로드
def load_tokens(token_file_path):
    with open(token_file_path, 'r', encoding='utf-8') as token_file:
        token_data = json.load(token_file)
    # 'token' 값을 사용하여 문자열 리스트를 생성
    processed_text = " ".join([item['token'] for item in token_data])
    return processed_text

# 챗봇 모델 초기화
model = ChatOpenAI(model="gpt-3.5-turbo")  # gpt-3.5-turbo, gpt-4 선택 가능

# 질문-응답 체인 로드
qa_chain = load_qa_chain(model, chain_type="map_reduce")

# 문서 분석 체인 생성
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

# 질문에 답변
def answer_question_using_tokens(question):
    # 토큰 로드
    processed_text = load_tokens(token_file_path)
    # 챗봇으로 질문 처리
    answer = qa_document_chain.run(
        input_document=processed_text,
        question=question
    )
    return answer

# 사용 예시


홈메이드 감자튀김을 만드는 방법은 다음과 같습니다:
1. 감자를 깨끗이 씻은 후 껍질을 벗기고 채썰어주세요.
2. 감자를 물에 살짝 삶아서 물기를 제거해주세요.
3. 감자를 바구니에 넣고 에어프라이어에 넣어주세요.
4. 감자튀김이 바삭하게 익을 때까지 에어프라이어에서 조리해주세요.
5. 완성된 감자튀김을 꺼내어 조금 식혀서 드시면 됩니다.

이렇게 간단하게 홈메이드 감자튀김을 만들 수 있습니다.


In [16]:
if __name__ == "__main__":
    question = "홈메이드 감자튀김 만드는 방법은?"
    answer = answer_question_using_tokens(question)
    print(answer)

홈메이드 감자튀김을 만드는 방법은 다음과 같습니다:

1. 감자를 깨끗이 세척한 후 껍질을 벗기고 채썰어주세요.
2. 감자를 물에 살짝 데친 뒤 물기를 제거해주세요.
3. 감자를 오븐이나 에어 프라이어에 넣고 골고루 익히세요.
4. 감자가 익으면 소금이나 기호에 맞는 양념을 넣고 섞어주세요.
5. 다시 오븐이나 에어 프라이어에 넣고 바삭하게 튀겨내면 완성입니다.
